Color moments

In [1]:
import cv2
import numpy as np
from scipy.stats import skew
import matplotlib.pyplot as plt

# === Funzione: Crop automatico dell'area informativa (cervello) ===
def crop_to_brain(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)
    x, y, w, h = cv2.boundingRect(coords)
    cropped = img[y:y+h, x:x+w]
    return cropped

# === Funzione: Estrazione Color Moments su griglia 10x10 ===
def extract_color_moments(img_path):
    print("[CM10x10] Estrazione in corso...")

    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"[ERRORE] Immagine non trovata: {img_path}")

    if len(img.shape) == 2 or img.shape[2] == 1:
        print("[CM10x10] Immagine in scala di grigi – conversione in RGB finto")
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    img = crop_to_brain(img)  # Crop dell'area utile
    img = cv2.resize(img, (300, 100))

    h, w, _ = img.shape
    grid_h, grid_w = h // 10, w // 10
    features = []

    for i in range(10):
        for j in range(10):
            cell = img[i*grid_h:(i+1)*grid_h, j*grid_w:(j+1)*grid_w]
            for channel in range(3):  # R, G, B
                pixels = cell[:, :, channel].flatten()
                if np.std(pixels) > 0:
                    mean = np.mean(pixels)
                    std = np.std(pixels)
                    sk = skew(pixels)
                    if np.isnan(sk):
                        sk = 0
                else:
                    mean, std, sk = 0, 0, 0
                features.extend([mean, std, sk])

    features = np.array(features)
    features = np.nan_to_num(features)

    print(f"[CM10x10] Feature vector (dim {features.shape[0]}): {features[:10]}")
    return features

# === Percorso immagine ===
img_path = "Part1/brain_glioma/brain_glioma_0001.jpg"
features = extract_color_moments(img_path)

[CM10x10] Estrazione in corso...
[CM10x10] Feature vector (dim 900): [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.36666667]


Task 2:

In [3]:
import cv2
import numpy as np
from scipy.stats import skew
import pandas as pd
import os

# === Funzione: Crop automatico dell'area informativa (cervello) ===
def crop_to_brain(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(thresh)
    if coords is not None:
        x, y, w, h = cv2.boundingRect(coords)
        cropped = img[y:y+h, x:x+w]
        return cropped
    else:
        return img

# === Funzione: Estrazione Color Moments su griglia 10x10 ===
def extract_color_moments(img_path):
    img = cv2.imread(img_path)
    if img is None:
        print(f"[ERRORE] Immagine non trovata: {img_path}")
        return None

    if len(img.shape) == 2 or img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    img = crop_to_brain(img)
    img = cv2.resize(img, (300, 100))

    h, w, _ = img.shape
    grid_h, grid_w = h // 10, w // 10
    features = []

    for i in range(10):
        for j in range(10):
            cell = img[i*grid_h:(i+1)*grid_h, j*grid_w:(j+1)*grid_w]
            for channel in range(3):
                pixels = cell[:, :, channel].flatten()
                if np.std(pixels) > 0:
                    mean = np.mean(pixels)
                    std = np.std(pixels)
                    sk = skew(pixels)
                    if np.isnan(sk):
                        sk = 0
                else:
                    mean, std, sk = 0, 0, 0
                features.extend([mean, std, sk])

    return features

# === Funzione: Elaborazione immagini in più sottocartelle ===
def process_multiple_folders(base_folder, subfolders):
    image_features = []

    for label in subfolders:
        folder_path = os.path.join(base_folder, label)
        print(f"[INFO] Elaboro cartella: {label}")

        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif')):
                img_path = os.path.join(folder_path, filename)
                print(f"  - Immagine: {filename}")
                features = extract_color_moments(img_path)
                if features is not None:
                    image_features.append({
                        "filename": filename,
                        "label": label,
                        "features": features
                    })

    return image_features

# === Esecuzione ===
base_folder = "Part1"
subfolders = ["brain_glioma", "brain_menin", "brain_tumor"]  # <-- Aggiungi le sottocartelle qui
image_features = process_multiple_folders(base_folder, subfolders)

# === Salvataggio in CSV ===
df = pd.DataFrame([{
    "filename": entry["filename"],
    "label": entry["label"],
    **{f"f{i}": val for i, val in enumerate(entry["features"])}
} for entry in image_features])

df.to_csv("color_moments_all.csv", index=False)
print("[SALVATO] Dati salvati in color_moments_all.csv")
print(f"[FINE] Elaborate {len(image_features)} immagini da {len(subfolders)} classi.")


[INFO] Elaboro cartella: brain_glioma
  - Immagine: brain_glioma_0001.jpg
  - Immagine: brain_glioma_0002.jpg
  - Immagine: brain_glioma_0003.jpg
  - Immagine: brain_glioma_0004.jpg
  - Immagine: brain_glioma_0005.jpg
  - Immagine: brain_glioma_0006.jpg
  - Immagine: brain_glioma_0007.jpg
  - Immagine: brain_glioma_0008.jpg
  - Immagine: brain_glioma_0009.jpg
  - Immagine: brain_glioma_0010.jpg
  - Immagine: brain_glioma_0011.jpg
  - Immagine: brain_glioma_0012.jpg
  - Immagine: brain_glioma_0013.jpg
  - Immagine: brain_glioma_0014.jpg
  - Immagine: brain_glioma_0015.jpg
  - Immagine: brain_glioma_0016.jpg
  - Immagine: brain_glioma_0017.jpg
  - Immagine: brain_glioma_0018.jpg
  - Immagine: brain_glioma_0019.jpg
  - Immagine: brain_glioma_0020.jpg
  - Immagine: brain_glioma_0021.jpg
  - Immagine: brain_glioma_0022.jpg
  - Immagine: brain_glioma_0023.jpg
  - Immagine: brain_glioma_0024.jpg
  - Immagine: brain_glioma_0025.jpg
  - Immagine: brain_glioma_0026.jpg
  - Immagine: brain_glioma